# Prerequisite

Following packages must be installed to run codes below

```bash
$ pip3 install openpyxl 
```


In [1]:
%load_ext autoreload
%autoreload 2

# Add project path

In [2]:
import os
import sys
from pathlib import Path

PROJECT_PATH = Path().resolve().parent.parent
SRC_PATH = PROJECT_PATH / 'src'

sys.path.append(str(SRC_PATH))

In [3]:
exp_name = 'feature_selection_v2'

# Load data

In [4]:
from data_loader import SectorDataLoader

data_loader = SectorDataLoader()
data_finance = (
    data_loader
    .load_dataset()
    .pipe(data_loader.add_beaver_indicator)
    .pipe(data_loader.add_label)
)
data_finance.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,...,STAT_OCR_DATE_y,Closed_Year,유동자산/부채총계,당기순이익(손실)/자산총계,부채총계/자산총계,순운전자본/자산총계,유동부채/유동자산,Years_From_Closed_Year_To_FS,Closed_In_1Yr,Closed_In_2Yrs
117,1018135422,(주),한국선박기술,NaN,NaN,Korea Marine Time Service,M,NaN,2,2,...,20200930.0,2020.0,1.031451,0.038773,0.578094,0.543045,0.089273,2.0,0,1
219,1018154206,(주),드림미즈,NaN,NaN,"dreammiz Co., Ltd.",M,NaN,2,2,...,20201231.0,2020.0,1.220040,0.001177,0.662356,0.608449,0.247063,2.0,0,1
279,1018163684,NaN,디에프에스서울,NaN,(주),DFS Seoul Ltd.,M,NaN,2,2,...,20190917.0,2019.0,11.012914,0.046666,0.090794,0.909114,0.090802,1.0,1,1
339,1018178760,(주),대교디앤씨,NaN,NaN,"DAEGYO D & C CO.,LTD.",M,NaN,2,2,...,20191010.0,2019.0,0.916364,-0.026128,1.091269,0.097514,0.902486,1.0,1,1
406,1018194173,NaN,에코에너지,NaN,(주),"Eco Energy Co.,Ltd.",M,NaN,2,2,...,20200831.0,2020.0,1.183355,-0.062095,0.828142,0.194625,0.801400,2.0,0,1


In [5]:
from data_loader import IndexDataLoader
import pandas as pd

index_features = [
    '3YEAR TREASURY',
    '5YEAR TREASURY',
    '10YEAR TREASURY',
    '3YEAR COPORATE BOND',
    'CD 91DAYS',
    'CALL RATE 1DAY',
    'BASE RATE',
    'YEAR_MONTH',
    'DOW_JONES_COMMODITY_INDEX_PRICE',
    'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE',
    'EXCHANGE_RATE_PRICE',
    'EXCHANGE_RATE_CHANGE_RATE',
    'NATIONWIDE',
    'CAPITAL AREA',
    'NON CAPITAL AREA',
    'CSI',
    'CCI',
    'CLI',
]

index_data_loader = IndexDataLoader()

index_data = (
    index_data_loader.load_interest_rate()
    .merge(
        (
            index_data_loader.load_dow_jones_commodity_index()
            .rename(columns={'PRICE':'DOW_JONES_COMMODITY_INDEX_PRICE',
                            'CHANGE RATE':'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE'})
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .merge(
        (
            index_data_loader.load_exchange_rate()
            .rename(columns={'PRICE':'EXCHANGE_RATE_PRICE',
                            'CHANGE RATE':'EXCHANGE_RATE_CHANGE_RATE'})
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .merge(
        (
            index_data_loader.load_apartment_housing_index()
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
        
    )
    .merge(
        (
            index_data_loader.load_csi()
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .merge(
        (
            index_data_loader.load_composite_index()
            .rename(columns={'PRICE':'DOW_JONES_COMMODITY_INDEX_PRICE',
                            'CHANGE RATE':'DOW_JONES_COMMODITY_INDEX_CHANGE_RATE'})
        ),
        how='outer',
        left_on='YEAR_MONTH',
        right_on='YEAR_MONTH'
    )
    .assign(year=lambda x : x['YEAR_MONTH'].dt.year)
    .assign(month=lambda x : x['YEAR_MONTH'].dt.month)
    .groupby(['year'])
    [index_features].mean()
    .reset_index(drop=False)
)

index_data

,year,3YEAR TREASURY,5YEAR TREASURY,10YEAR TREASURY,3YEAR COPORATE BOND,CD 91DAYS,CALL RATE 1DAY,BASE RATE,DOW_JONES_COMMODITY_INDEX_PRICE,EXCHANGE_RATE_PRICE,NATIONWIDE,CAPITAL AREA,NON CAPITAL AREA,CSI,CCI,CLI
0,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.841667,68.600000,58.416667,NaN,NaN,NaN
1,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.758333,82.441667,60.925000,NaN,NaN,NaN
2,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.775000,89.150000,62.991667,NaN,NaN,NaN
3,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.058333,87.833333,64.200000,NaN,NaN,NaN
4,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.733333,86.833333,69.516667,NaN,NaN,NaN
5,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.700000,86.850000,80.983333,NaN,NaN,NaN
6,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.883333,81.891667,85.250000,NaN,NaN,NaN
7,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.325000,80.333333,87.125000,NaN,NaN,NaN
8,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.008333,83.366667,91.325000,NaN,NaN,NaN
9,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.008333,89.175000,97.658333,NaN,NaN,NaN


In [6]:
data = (
    data_finance
    .assign(year=lambda x : x['결산년월'].astype(str).str[:4].astype(int))
    .merge(
        index_data,
        how='left',
        left_on='year',
        right_on='year'
    )
)
data

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,...,CALL RATE 1DAY,BASE RATE,DOW_JONES_COMMODITY_INDEX_PRICE,EXCHANGE_RATE_PRICE,NATIONWIDE,CAPITAL AREA,NON CAPITAL AREA,CSI,CCI,CLI
0,1018135422,(주),한국선박기술,NaN,NaN,Korea Marine Time Service,M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
1,1018154206,(주),드림미즈,NaN,NaN,"dreammiz Co., Ltd.",M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
2,1018163684,NaN,디에프에스서울,NaN,(주),DFS Seoul Ltd.,M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
3,1018178760,(주),대교디앤씨,NaN,NaN,"DAEGYO D & C CO.,LTD.",M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
4,1018194173,NaN,에코에너지,NaN,(주),"Eco Energy Co.,Ltd.",M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10516,8998700283,(주),덕성건설,NaN,NaN,Deokseong Construction,M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
10517,8998700283,(주),덕성건설,NaN,NaN,Deokseong Construction,M,NaN,2,2,...,1.591667,1.562500,614.237500,1164.787500,100.108333,107.183333,94.075000,99.083333,100.258333,98.475000
10518,8998701263,NaN,하나머스트제6호기업인수목적,NaN,(주),HANA MUST SIXTH SPECIAL PURPOSE ACQUISITION CO...,M,NaN,2,2,...,1.521667,1.541667,625.940833,1098.985000,100.608333,104.325000,97.441667,104.083333,101.425000,100.058333
10519,8998701263,NaN,하나머스트제6호기업인수목적,NaN,(주),HANA MUST SIXTH SPECIAL PURPOSE ACQUISITION CO...,M,NaN,2,2,...,1.591667,1.562500,614.237500,1164.787500,100.108333,107.183333,94.075000,99.083333,100.258333,98.475000


# Baseline classifier using Beaver's features

In [7]:
from project_paths import DATA_PATH

sectors = [
    '제조업(10~34)',
    '부동산업(68)',
    '도매 및 소매업(45~47)',
    '숙박 및 음식점업(55~56)',
    '건설업(41~42)'
]

beaver_features = [
    '유동자산/부채총계',
    '당기순이익(손실)/자산총계',
    '부채총계/자산총계',
    '순운전자본/자산총계',
    '유동부채/유동자산'
]

extended_financial_features = [
    '유동자산',
    '매출채권',
    '비유동자산',
    '유형자산',
    '자산총계',
    '유동부채',
    '비유동부채',
    '부  채  총  계',
    '자본금',
    '이익잉여금(결손금）',
    '자본총계',
    '매출액',
    '판매비와관리비',
    '영업이익（손실）',
    '법인세비용차감전순손익',
    '법인세비용',
    '당기순이익(손실)',
    '기업순이익률(%)',
    '유보액/총자산(%)',
    '유보액/납입자본(%)',
    '매출액총이익률(%)',
    '매출액영업이익률(%)',
    '매출액순이익률(%)',
    '수지비율(%)',
    '경상수지비율',
    '영업비율(%)',
    '금융비용대매출액비율(%',
    '금융비용대부채비율(%)',
    '금융비용대총비용비율(%',
    '부채비율(%)',
    '차입금의존도(%)',
    '자기자본비율(%)',
    '순운전자본비율(%)',
    '유동부채비율(%)',
    '비유동부채비율(%)',
    '부채총계대 매출액(%)',
    '총자본회전율(회)',
    '재고자산회전율(회)',
    '매출채권회전율(회)',
    '매입채무회전율(회)',
    '미수금',
    '매출원가',
    '무형자산',
    '재고자산',
]

years_to_close = [
    'Closed_In_1Yr',
    'Closed_In_2Yrs',
]

x_data_year = list(range(2018, 2020))

EXP_RESULT_PATH = DATA_PATH / 'experiment_result' / exp_name

if not os.path.exists(EXP_RESULT_PATH) : 
    os.mkdir(EXP_RESULT_PATH)

In [8]:
import numpy as np
import pandas as pd
from pycaret.classification import ClassificationExperiment
from tqdm import tqdm

In [9]:
exp_result = None

for target_years_to_close in tqdm(years_to_close) : 
    for x_data_target_year in tqdm(x_data_year, leave=True) : 
        for sector in sectors : 

            filename = EXP_RESULT_PATH / f'Beaver_baseline_{sector}_{target_years_to_close}_{x_data_target_year}.csv'

            if not os.path.exists(filename) : 

                data_to_train = (
                    data
                    .loc[~data[beaver_features[0]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[1]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[2]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[3]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[4]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[lambda x : pd.to_datetime(x['결산년월'], format='%Y%m%d').dt.year==x_data_target_year]
                    .loc[lambda x : x['대분류']==sector]
                )

                exp = ClassificationExperiment()

                exp.setup(
                    (
                        data_to_train
                        .loc[:, beaver_features
                             +extended_financial_features
                             +index_data.drop(['year'], axis=1).columns.tolist()
                             +[target_years_to_close]]
                        .reset_index(drop=True)
                    ),
                    target=target_years_to_close,
                    feature_selection=True,
                    #remove_multicollinearity = True,
                )

                models = exp.compare_models(sort='AUC', n_select=3)
                tuned_models = [exp.tune_model(x) for x in models]
                
                #if target_years_to_close == 'Closed_In_1Yr' :
                blended_model = exp.blend_models(estimator_list=tuned_models, fold=5, method = 'soft')
                result = (
                    exp.pull()
                    .assign(target=target_years_to_close)
                    .assign(x_data_yaer=x_data_target_year)
                    .assign(대분류=sector)
                    .reset_index(drop=False)
                    .rename(columns={'index':'model_name'})
                )
                result['feature_selection'] = str(exp.dataset_transformed.columns.tolist())
                result.to_csv(filename)

            else : 
                result = pd.read_csv(filename)

            if exp_result is None : 
                exp_result = result
            else : 
                exp_result = pd.concat([exp_result, result], axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 55.90it/s]

  0%|                                                                                    | 0/2 [00:00<?, ?it/s]

,Description,Value
0,Session id,8903
1,Target,Closed_In_2Yrs
2,Target type,Binary
3,Original data shape,"(1800, 65)"
4,Transformed data shape,"(1800, 11)"
5,Transformed train set shape,"(1259, 11)"
6,Transformed test set shape,"(541, 11)"
7,Numeric features,64
8,Rows with missing values,84.8%
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5799,0.6005,0.4991,0.5391,0.5171,0.1467,0.1474,0.0310
et,Extra Trees Classifier,0.5782,0.5981,0.4238,0.5462,0.4769,0.1328,0.1361,0.0800
rf,Random Forest Classifier,0.5870,0.5948,0.4081,0.5639,0.4729,0.1472,0.1530,0.0960
gbc,Gradient Boosting Classifier,0.5758,0.5894,0.4062,0.5436,0.4635,0.1254,0.1294,0.0670
ada,Ada Boost Classifier,0.5616,0.5714,0.4201,0.5203,0.4637,0.1005,0.1023,0.0390
dt,Decision Tree Classifier,0.5552,0.5607,0.5445,0.5110,0.5266,0.1082,0.1084,0.0130
knn,K Neighbors Classifier,0.5393,0.5402,0.4257,0.4904,0.4547,0.0599,0.0605,0.0230
lr,Logistic Regression,0.4877,0.5168,0.6707,0.4563,0.5415,0.0066,0.0017,0.3340
lda,Linear Discriminant Analysis,0.5322,0.5151,0.0753,0.4300,0.1272,-0.0143,-0.0199,0.0130
nb,Naive Bayes,0.5449,0.5107,0.0753,0.5170,0.1298,0.0108,0.0260,0.0130


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6190,0.5852,0.4828,0.6087,0.5385,0.2214,0.2257
1,0.5873,0.5947,0.4211,0.5581,0.4800,0.1489,0.1529
2,0.5873,0.5299,0.3860,0.5641,0.4583,0.1435,0.1503
3,0.6111,0.6641,0.4386,0.5952,0.5051,0.1967,0.2030
4,0.5794,0.6112,0.5088,0.5370,0.5225,0.1471,0.1473
5,0.5556,0.5795,0.4035,0.5111,0.4510,0.0862,0.0879
6,0.5794,0.5840,0.4561,0.5417,0.4952,0.1392,0.1407
7,0.6508,0.5978,0.4386,0.6757,0.5319,0.2730,0.2893
8,0.5476,0.5327,0.4035,0.5000,0.4466,0.0714,0.0725


Fitting 10 folds for each of 10 candidates, totalling 100 fits


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7143,0.6500,1.0000,0.7143,0.8333,0.0000,0.0000
1,0.7143,0.7222,1.0000,0.7143,0.8333,0.0000,0.0000
2,0.7619,0.6667,1.0000,0.7500,0.8571,0.2222,0.3536
3,0.7143,0.7111,1.0000,0.7143,0.8333,0.0000,0.0000
4,0.7143,0.6667,1.0000,0.7143,0.8333,0.0000,0.0000
5,0.7500,0.6267,0.9333,0.7778,0.8485,0.1667,0.1925
6,0.7500,0.9333,1.0000,0.7500,0.8571,0.0000,0.0000
7,0.7500,0.3933,1.0000,0.7500,0.8571,0.0000,0.0000
8,0.7000,0.5467,0.9333,0.7368,0.8235,-0.0909,-0.1325


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7317,0.4455,0.9333,0.7568,0.8358,0.1442,0.1719
1,0.6585,0.7212,0.8333,0.7353,0.7812,0.0171,0.0178
2,0.7073,0.4864,0.9667,0.7250,0.8286,-0.0468,-0.0957
3,0.7317,0.7333,1.0000,0.7317,0.8451,0.0000,0.0000
4,0.5366,0.4756,0.5862,0.7083,0.6415,0.0026,0.0027
Mean,0.6732,0.5724,0.8639,0.7314,0.7864,0.0234,0.0193
Std,0.0733,0.1272,0.1496,0.0157,0.0757,0.0641,0.0862



100%|███████████████████████████████████████████████████████████████████████████| 2/2 [04:16<00:00, 128.50s/it]


In [10]:
aggregated_result_path = EXP_RESULT_PATH / 'result.csv'

if not os.path.exists(aggregated_result_path) :
    exp_result.to_csv(aggregated_result_path)
    
exp_result

,Unnamed: 0,Fold,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,target,x_data_yaer,대분류,feature_selection
0,0.0,0,0.8690,0.6121,0.0000,0.0000,0.0000,0.0000,0.0000,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
1,1.0,1,0.8690,0.6165,0.0000,0.0000,0.0000,0.0000,0.0000,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
2,2.0,2,0.8690,0.7103,0.0000,0.0000,0.0000,0.0000,0.0000,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
3,3.0,3,0.8690,0.5995,0.0000,0.0000,0.0000,0.0000,0.0000,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
4,4.0,4,0.8725,0.6568,0.0000,0.0000,0.0000,0.0000,0.0000,Closed_In_1Yr,2018,제조업(10~34),"['순운전자본/자산총계', '매출채권', '비유동자산', '유형자산', '자본금',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,NaN,2,0.7073,0.4864,0.9667,0.7250,0.8286,-0.0468,-0.0957,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
3,NaN,3,0.7317,0.7333,1.0000,0.7317,0.8451,0.0000,0.0000,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
4,NaN,4,0.5366,0.4756,0.5862,0.7083,0.6415,0.0026,0.0027,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
5,NaN,Mean,0.6732,0.5724,0.8639,0.7314,0.7864,0.0234,0.0193,Closed_In_2Yrs,2019,건설업(41~42),"['매출채권', '비유동자산', '유형자산', '판매비와관리비', '기업순이익률(%..."
